In [44]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json
import cx_Oracle
import os
import pandas as pd


os.environ['TNS_ADMIN'] = '/home/opc/adj_esportsdb'

In [45]:
import os

os.environ['PATH']='/home/ubuntu/miniconda3/bin:$PATH'

In [46]:
!pip install simplejson
!pip install dataprep
!pip install dask
!pip install pandas_profiling
## install packages
!pip install -q scikit-learn
!pip install -U setuptools wheel
!pip install -U "mxnet<2.0.0"
!pip install autogluon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.3/130.3 KB 27.0 MB/s eta 0:00:00
You should consider upgrading via the '/home/ubuntu/miniconda3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/ubuntu/miniconda3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/ubuntu/miniconda3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/ubuntu/miniconda3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/ubuntu/miniconda3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/ubuntu/miniconda3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/ubuntu/miniconda3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/ubuntu/miniconda3/bin/python -m pip install --upgrade pip' command.


In [47]:
import cx_Oracle
import yaml
import os
from pathlib import Path
home = str(Path.home())

def process_yaml():
	with open("../config.yaml") as file:
		return yaml.safe_load(file)


class OracleJSONDatabaseConnection:
    def __init__(self, data=process_yaml()):
        # wallet location (default is HOME/wallets/wallet_X)
        os.environ['TNS_ADMIN'] = '{}/{}'.format(home, process_yaml()['WALLET_DIR'])
        print(os.environ['TNS_ADMIN'])
        self.pool = cx_Oracle.SessionPool(data['db']['username'], data['db']['password'], data['db']['dsn'],
            min=1, max=4, increment=1, threaded=True,
            getmode=cx_Oracle.SPOOL_ATTRVAL_WAIT
        )
        print('Connection successful.')



    def close_pool(self):
        self.pool.close()
        print('Connection pool closed.')



    def insert(self, collection_name, json_object_to_insert):
        connection = self.pool.acquire()
        connection.autocommit = True
        soda = connection.getSodaDatabase()
        x_collection = soda.createCollection(collection_name)

        try:
            x_collection.insertOne(json_object_to_insert)
            print('[DBG] INSERT {} OK'.format(json_object_to_insert))
        except cx_Oracle.IntegrityError:
            print('[DBG] INSERT {} ERR'.format(json_object_to_insert))
            return 0
        self.pool.release(connection)
        return 1



    def delete(self, collection_name, on_column, on_value):
        connection = self.pool.acquire()
        soda = connection.getSodaDatabase()
        x_collection = soda.createCollection(collection_name)
        qbe = {on_column: on_value}
        x_collection.find().filter(qbe).remove()
        self.pool.release(connection)



    def get_connection(self):
        return self.pool.acquire() 



    def close_connection(self, conn_object):
        self.pool.release(conn_object)



    def get_collection_names(self):
        connection = self.pool.acquire()
        returning_object = connection.getSodaDatabase().getCollectionNames(startName=None, limit=0)
        self.pool.release(connection)
        return returning_object



    def open_collection(self, collection_name):
        connection = self.pool.acquire()
        returning_object = self.pool.acquire().getSodaDatabase().openCollection(collection_name)
        self.pool.release(connection)
        return returning_object



def test_class():
    object = OracleJSONDatabaseConnection()
    print(object.pool)
    object.close_pool()

In [48]:
print(os.environ['TNS_ADMIN'])

/home/opc/adj_esportsdb


In [49]:
db = OracleJSONDatabaseConnection()
print(db.get_collection_names())

/home/ubuntu/wallets/Wallet_eSportsDB
Connection successful.
['1v1_model', 'match', 'match_detail', 'matchups', 'predictor', 'predictor_liveclient', 'summoner']


In [50]:
# monkeypatch using faster simplejson module
import simplejson
pd.io.json._json.loads = lambda s, *a, **kw: simplejson.loads(s)

In [51]:
data = db.open_collection('predictor_liveclient')
all_data = list()
i = 0
for doc in data.find().getCursor():
    content = doc.getContent()
    all_data.append(content)
    i+= 1
    if i > 500000:
        break

print('Data length: {}'.format(len(all_data)))

Data length: 500001


In [52]:
df = pd.read_json(json.dumps(all_data), orient='records')

df.head(5)

,winner,magicResist,healthRegenRate,spellVamp,timestamp,bonusArmorPenetrationPercent,bonusMagicPenetrationPercent,maxHealth,identifier,moveSpeed,...,cooldownReduction,resourceValue,magicPenetrationFlat,attackSpeed,currentHealth,armor,magicPenetrationPercent,armorPenetrationFlat,resourceMax,resourceRegenRate
0,1,30,11,0,60025,0,0,520,BR1_2267584388_5,697,...,0,469,0,100,520,28,0,0,469,16
1,0,32,17,0,60025,0,0,660,BR1_2267584388_6,335,...,0,350,0,110,660,42,0,0,350,16
2,0,30,11,0,60025,0,0,560,BR1_2267584388_7,340,...,0,340,0,100,560,31,0,0,340,22
3,0,32,14,0,60025,0,0,584,BR1_2267584388_8,345,...,0,180,0,100,584,38,0,0,200,100
4,0,30,7,0,60025,0,0,680,BR1_2267584388_9,335,...,0,344,0,110,592,34,0,0,344,16


In [25]:
df = df.fillna(0)

In [26]:
df.describe()

,winner,magicResist,healthRegenRate,spellVamp,timestamp,bonusArmorPenetrationPercent,bonusMagicPenetrationPercent,maxHealth,moveSpeed,attackDamage,...,cooldownReduction,resourceValue,magicPenetrationFlat,attackSpeed,currentHealth,armor,magicPenetrationPercent,armorPenetrationFlat,resourceMax,resourceRegenRate
count,500001.000000,500001.000000,500001.000000,500001.000000,5.000010e+05,500001.0,500001.0,500001.000000,500001.000000,500001.000000,...,500001.0,500001.000000,500001.000000,500001.000000,500001.000000,500001.000000,500001.000000,500001.0,500001.000000,500001.00000
mean,0.499969,45.202852,42.668093,0.054268,8.641028e+05,0.0,0.0,1373.862534,382.182052,122.361233,...,0.0,690.072270,2.247516,141.004362,1018.197318,69.115016,0.565651,0.0,879.129072,42.20194
std,0.500000,21.270119,92.360539,0.985789,5.588095e+05,0.0,0.0,619.323108,60.230610,75.520146,...,0.0,1265.138271,6.925615,44.513022,671.986160,38.627498,3.703696,0.0,1294.995514,49.54758
min,0.000000,-10.000000,0.000000,0.000000,0.000000e+00,0.0,0.0,340.000000,0.000000,1.000000,...,0.0,0.000000,0.000000,50.000000,0.000000,-14.000000,0.000000,0.0,0.000000,0.00000
25%,0.000000,33.000000,16.000000,0.000000,4.200960e+05,0.0,0.0,889.000000,345.000000,73.000000,...,0.0,245.000000,0.000000,112.000000,562.000000,45.000000,0.000000,0.0,400.000000,17.00000
50%,0.000000,38.000000,23.000000,0.000000,8.402240e+05,0.0,0.0,1291.000000,375.000000,93.000000,...,0.0,441.000000,0.000000,125.000000,946.000000,59.000000,0.000000,0.0,636.000000,26.00000
75%,1.000000,47.000000,35.000000,0.000000,1.260431e+06,0.0,0.0,1720.000000,395.000000,154.000000,...,0.0,743.000000,0.000000,154.000000,1437.000000,81.000000,0.000000,0.0,948.000000,48.00000
max,1.000000,492.000000,5682.000000,37.000000,3.147362e+06,0.0,0.0,7019.000000,1224.000000,936.000000,...,0.0,10000.000000,50.000000,621.000000,6921.000000,847.000000,48.000000,0.0,10000.000000,694.00000


In [27]:
from pandas_profiling import ProfileReport

In [28]:
#report = ProfileReport(df)
#report #uncomment to display all.

In [29]:
from autogluon.tabular import TabularPredictor, TabularDataset

In [30]:
df = TabularDataset(df)

# drop columns we don't want (constant values + identifier)
df = df.drop(columns=['bonusArmorPenetrationPercent', 'bonusMagicPenetrationPercent',
    'identifier', 'cooldownReduction', 'armorPenetrationFlat'])

train = df.sample(frac=0.8,random_state=200) #random state is a seed value
test = df.drop(train.index)

train.head(5)

,winner,magicResist,healthRegenRate,spellVamp,timestamp,maxHealth,moveSpeed,attackDamage,armorPenetrationPercent,lifesteal,abilityPower,resourceValue,magicPenetrationFlat,attackSpeed,currentHealth,armor,magicPenetrationPercent,resourceMax,resourceRegenRate
461559,0,30,11,0,180049,662,340,57,0,0,33,270,0,100,496,30,0,549,17
38055,1,66,136,0,600233,1265,392,186,20,0,0,583,0,115,720,59,0,650,80
46195,1,31,13,0,360086,759,335,58,0,0,26,290,0,105,440,40,0,712,42
454169,0,33,20,0,960241,1144,390,67,0,0,112,910,0,115,1144,53,0,1120,97
373019,1,44,95,0,1020349,2096,483,212,0,0,0,0,0,124,1428,130,0,0,0


In [31]:
df.describe()

,winner,magicResist,healthRegenRate,spellVamp,timestamp,maxHealth,moveSpeed,attackDamage,armorPenetrationPercent,lifesteal,abilityPower,resourceValue,magicPenetrationFlat,attackSpeed,currentHealth,armor,magicPenetrationPercent,resourceMax,resourceRegenRate
count,500001.000000,500001.000000,500001.000000,500001.000000,5.000010e+05,500001.000000,500001.000000,500001.000000,500001.000000,500001.000000,500001.000000,500001.000000,500001.000000,500001.000000,500001.000000,500001.000000,500001.000000,500001.000000,500001.00000
mean,0.499969,45.202852,42.668093,0.054268,8.641028e+05,1373.862534,382.182052,122.361233,1.019892,1.471075,49.147090,690.072270,2.247516,141.004362,1018.197318,69.115016,0.565651,879.129072,42.20194
std,0.500000,21.270119,92.360539,0.985789,5.588095e+05,619.323108,60.230610,75.520146,5.082061,4.182162,96.778322,1265.138271,6.925615,44.513022,671.986160,38.627498,3.703696,1294.995514,49.54758
min,0.000000,-10.000000,0.000000,0.000000,0.000000e+00,340.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,50.000000,0.000000,-14.000000,0.000000,0.000000,0.00000
25%,0.000000,33.000000,16.000000,0.000000,4.200960e+05,889.000000,345.000000,73.000000,0.000000,0.000000,0.000000,245.000000,0.000000,112.000000,562.000000,45.000000,0.000000,400.000000,17.00000
50%,0.000000,38.000000,23.000000,0.000000,8.402240e+05,1291.000000,375.000000,93.000000,0.000000,0.000000,0.000000,441.000000,0.000000,125.000000,946.000000,59.000000,0.000000,636.000000,26.00000
75%,1.000000,47.000000,35.000000,0.000000,1.260431e+06,1720.000000,395.000000,154.000000,0.000000,0.000000,53.000000,743.000000,0.000000,154.000000,1437.000000,81.000000,0.000000,948.000000,48.00000
max,1.000000,492.000000,5682.000000,37.000000,3.147362e+06,7019.000000,1224.000000,936.000000,58.000000,125.000000,1260.000000,10000.000000,50.000000,621.000000,6921.000000,847.000000,48.000000,10000.000000,694.00000


In [32]:
label = 'winner'

In [33]:
save_path = './autogluon_trained_models_liveclient_classifier'  # specifies folder to store trained models
predictor = TabularPredictor(label=label, path=save_path).fit(train)

	Consider setting `time_limit` to ensure training finishes within an expected duration or experiment with a small portion of `train_data` to identify an ideal `presets` and `hyperparameters` configuration.
Beginning AutoGluon training ...
AutoGluon will save models to "./autogluon_trained_models_liveclient_classifier/"
AutoGluon Version:  0.3.1
Train Data Rows:    400001
Train Data Columns: 18
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [0, 1]
	If 'binary' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    262455.58 MB
	Train Data (Original)  Memory Usage: 57.6 MB (0.0% o

In [34]:
y_test = test[label]  # values to predict
test_data_nolabel = test.drop(columns=[label])  # delete label column to prove we're not cheating, also drop identifier column
test_data_nolabel.head(5)

,magicResist,healthRegenRate,spellVamp,timestamp,maxHealth,moveSpeed,attackDamage,armorPenetrationPercent,lifesteal,abilityPower,resourceValue,magicPenetrationFlat,attackSpeed,currentHealth,armor,magicPenetrationPercent,resourceMax,resourceRegenRate
0,32,12,0,660207,1038,370,166,0,4,0,112,0,190,752,51,0,590,17
2,39,23,0,720219,1215,375,139,0,0,0,599,0,131,1215,86,0,599,23
8,118,20,0,720219,1375,208,76,0,0,9,0,0,107,520,122,0,0,0
9,88,40,0,1380459,2688,427,101,0,0,0,0,0,117,0,180,0,0,0
13,36,22,0,1380459,1449,375,72,0,0,144,1008,0,129,0,71,0,1232,113


In [35]:
predictor = TabularPredictor.load(save_path)

y_pred = predictor.predict(test_data_nolabel)
print("Predictions:  \n", y_pred)
#perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)


Predictions:  
 0         1
2         0
8         1
9         0
13        1
         ..
499986    1
499990    1
499993    0
499995    0
499999    0
Name: winner, Length: 100000, dtype: int64


In [36]:
predictor.leaderboard(test, silent=False)

                  model  score_test  score_val  pred_time_test  pred_time_val    fit_time  pred_time_test_marginal  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0   WeightedEnsemble_L2     0.66544   0.689078       12.998381       1.530008  866.161298                 0.006004                0.006177           1.356416            2       True         14
1      RandomForestGini     0.66069   0.676831        2.016773       0.315675   22.368041                 2.016773                0.315675          22.368041            1       True          5
2      RandomForestEntr     0.66011   0.677081        1.908432       0.314932   24.854358                 1.908432                0.314932          24.854358            1       True          6
3        ExtraTreesGini     0.65994   0.669833        4.162963       0.314386   11.347579                 4.162963                0.314386          11.347579            1       True          8
4        ExtraTreesEntr     0.65928

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.66544,0.689078,12.998381,1.530008,866.161298,0.006004,0.006177,1.356416,2,True,14
1,RandomForestGini,0.66069,0.676831,2.016773,0.315675,22.368041,2.016773,0.315675,22.368041,1,True,5
2,RandomForestEntr,0.66011,0.677081,1.908432,0.314932,24.854358,1.908432,0.314932,24.854358,1,True,6
3,ExtraTreesGini,0.65994,0.669833,4.162963,0.314386,11.347579,4.162963,0.314386,11.347579,1,True,8
4,ExtraTreesEntr,0.65928,0.673332,3.386951,0.313787,11.595320,3.386951,0.313787,11.595320,1,True,9
5,NeuralNetMXNet,0.64286,0.648588,1.480003,0.242218,793.445433,1.480003,0.242218,793.445433,1,True,12
6,LightGBMLarge,0.60645,0.622844,0.079303,0.015620,3.809432,0.079303,0.015620,3.809432,1,True,13
7,NeuralNetFastAI,0.60542,0.615846,7.990198,0.051421,325.652130,7.990198,0.051421,325.652130,1,True,10
8,XGBoost,0.59687,0.614096,0.070154,0.033291,0.968276,0.070154,0.033291,0.968276,1,True,11
9,LightGBM,0.56626,0.575606,0.022063,0.011376,0.475452,0.022063,0.011376,0.475452,1,True,4


In [37]:
predictor.feature_importance(test)

Computing feature importance via permutation shuffling for 18 features using 1000 rows with 3 shuffle sets...
	549.57s	= Expected runtime (183.19s per shuffle set)
	62.24s	= Actual runtime (Completed 3 of 3 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
timestamp,0.115667,0.015044,0.002796,3,0.201873,0.029461
attackDamage,0.090333,0.018610,0.006927,3,0.196971,-0.016304
abilityPower,0.069000,0.009000,0.002812,3,0.120571,0.017429
maxHealth,0.044667,0.025541,0.046946,3,0.191018,-0.101685
armor,0.040667,0.021502,0.040951,3,0.163875,-0.082542
resourceMax,0.038667,0.009713,0.010195,3,0.094321,-0.016987
magicResist,0.037333,0.013650,0.020895,3,0.115552,-0.040885
resourceValue,0.030667,0.024028,0.078814,3,0.168348,-0.107015
attackSpeed,0.028667,0.022301,0.077944,3,0.156454,-0.099120
resourceRegenRate,0.027333,0.015503,0.046289,3,0.116165,-0.061499


In [38]:
data = [91, 35, 0, 1080321, 2204, 390, 225, 0, 10, 0, 672, 0, 220, 0, 94, 0, 1047, 33]

test_d = pd.DataFrame([data], columns=['magicResist', 'healthRegenRate', 'spellVamp', 'timestamp', 'maxHealth', 'moveSpeed', 'attackDamage', 'armorPenetrationPercent', 'lifesteal', 'abilityPower', 'resourceValue', 'magicPenetrationFlat', 'attackSpeed', 'currentHealth', 'armor', 'magicPenetrationPercent', 'resourceMax', 'resourceRegenRate'])

In [39]:
predictor.predict(test_d)

0    1
Name: winner, dtype: int64

In [40]:
pred_probs = predictor.predict_proba(test_d)

In [41]:
print(pred_probs)

          0         1
0  0.205442  0.794558
